In [319]:
import pickle
import pandas as pd
import numpy as np

In [320]:
file = open("out.pickle", "rb")
test = pickle.load(file)

In [321]:
threshold = 0.55

In [322]:
keep = []
for box in test:
    if box["score"] >= threshold:
        box["x1"] = box["bbox"][0]
        box["y1"] = box["bbox"][1]
        box["x2"] = box["bbox"][2]
        box["y2"] = box["bbox"][3]
        del box["category_id"]
        del box["bbox"]
        keep.append(box)

In [323]:
df = pd.DataFrame(keep)

In [324]:
df.dtypes

category     object
image_id     object
score       float64
x1          float64
x2          float64
y1          float64
y2          float64
dtype: object

In [325]:
def get_number(df):
    df = df.sort_values(by=["x1"])
    score = ""
    for index, row in df.iterrows():
        score += str(row["category"])
    k = 0
    # print(score)
    while k < len(score) - 1 :
        try:
            if score[k] == "." and score[k + 1] == ".":
                score = score[:k] + score[k + 1:]
                k -= 1
        except Exception as e:
            print(score, k)
            raise e 
        k += 1
    return float(score)

scores = df.groupby("image_id").apply(get_number)
scores = scores.reset_index()
scores.columns = ["image", "used_liters_predict"]
scores.head()

,image,used_liters_predict
0,Datasets/img/005fc43f13dfef916914b17ef7ec7deb9...,376.01
1,Datasets/img/00e22973fdbc19f1f703cdbd05a860670...,230.00
2,Datasets/img/0122de9a0c734ad199c00b352cc3b1272...,25.42
3,Datasets/img/0133c36547f01438b1803e928af3719f1...,50.00
4,Datasets/img/015538fa03041bcfdbb196e8eec8a0211...,136.00


In [326]:
path_useless = "Datasets/img/"
scores["image"] = scores.image.str[len(path_useless):]
scores.head()

,image,used_liters_predict
0,005fc43f13dfef916914b17ef7ec7deb932bb33b.jpg,376.01
1,00e22973fdbc19f1f703cdbd05a8606705bc60d1.jpg,230.00
2,0122de9a0c734ad199c00b352cc3b12729238b44.jpg,25.42
3,0133c36547f01438b1803e928af3719f13c95487.jpg,50.00
4,015538fa03041bcfdbb196e8eec8a0211e52201d.jpg,136.00


In [327]:
hq_dig = pd.read_csv("Datasets/HQ_quality.csv", sep=";", index_col=0)
hq_ana = pd.read_csv("Datasets/HQ_analog.csv", sep=";", index_col=0)
lq_dig = pd.read_csv("Datasets/LQ_digital.csv", sep=";", index_col=0)
lq_ana = pd.read_csv("Datasets/LQ_analog.csv", sep=";", index_col=0)
mq_dig = pd.read_csv("Datasets/MQ_digital.csv", sep=";", index_col=0)

hq_dig["q"] = "HD"
hq_ana["q"] = "HA"
lq_dig["q"] = "LD"
lq_ana["q"] = "LA"
mq_dig["q"] = "MD"
ground_truth = pd.concat((hq_dig, hq_ana, lq_dig, lq_ana, mq_dig))

In [328]:
ground_truth.head()

,used_liter,image,q
0,33,e104664ba1792dde641d87cd5d95f1df06786140.jpg,HD
1,29,48990b5cbe173868040bd33f06fb1b80c2b4f28a.jpg,HD
2,230,9e111802446b62b86aeffe911415ad28227caba7.jpg,HD
3,43,f78fe5a461f28bc770a7dd856878bb4a314a9027.jpg,HD
4,51,62e96bf40409b3b6fd74065ab0bf3ec44f9d0a61.jpg,HD


In [329]:
final = scores.merge(ground_truth, on="image")

In [330]:
final.head()

,image,used_liters_predict,used_liter,q
0,005fc43f13dfef916914b17ef7ec7deb932bb33b.jpg,376.01,376,MD
1,00e22973fdbc19f1f703cdbd05a8606705bc60d1.jpg,230.00,230,MD
2,0122de9a0c734ad199c00b352cc3b12729238b44.jpg,25.42,25,MD
3,0133c36547f01438b1803e928af3719f13c95487.jpg,50.00,50,MD
4,015538fa03041bcfdbb196e8eec8a0211e52201d.jpg,136.00,136,MD


In [331]:
final["correct"] = np.abs(final["used_liters_predict"] - final["used_liter"])

In [332]:
final["bool"] = final["correct"] <= 1

In [333]:
gb = final.groupby("q")
total = gb["bool"].count()

In [334]:
gb2 = final[final["bool"] == True].groupby("q")
total2 = gb2["bool"].count()

In [335]:
total2/total

q
HA    0.507463
HD    0.895833
MD    0.963636
Name: bool, dtype: float64